## <font color='#FBD5AB'>Agenda</font>

### - Objective and its Importance

### - Datasets Used 

### - Understand the data and Feature Selection

### - Model Building. Why choose LSTM?

### - Next Steps

## <font color='#FBD5AB'>OBJECTIVE: Predict Oil Prices</font>

Brent Oil Prices international benchmark for crude oil prices and predicting these prices can be extremely useful.


- `Economic Planning`
- `Mitigate Financial Risk`
- `Improved Investment Desicions`

However, achieving this task is difficult due to its high volatitlity and the effect of macroeconomic variables. This analysis aims to predict the future oil prices while taking into account this uncertainity. 


## <font color='#FBD5AB'>DATASETS</font>

The features/variables we will be using are:

- `Brent Oil Prices`

- `Supply Estimates` (Source: US Energy Information Administration)
  - `OPEC Total Production Capacity`
  - `OPEC Total Production of Oil`
  - `Saudi Arabia's oil production` 
  - `OPEC Surplus Total Capacity` 



- `Demand Estimates` (Source: US Energy Information Administration)
  - `Total World Petroleum Liquids Consumption`
  - `Total Non-OECD Liquid fuels Consumption`
  - `Total OECD Liquid fuels Consumption`
  - `Non OPEC Liquid Petroleum Production` 

- `Financial Market` (Source: Investings.com) 
  - `Brent Futures`  
  - `Bent Oil and Future Price Difference for the same day` 
  - `Trade Volume` 

- `US Gasoline Prices` (Source: Federal Reserve Economic Data)

- `Macroeconomic Variables`
  - `Federal Funds Effective Rate` (Source: Federal Reserve Economic Data)
  - `Inflation Indicators` (source: World Bank Data)
    - `HCPI`
    - `EPI`  
  - `World GDP`  (Source: CCI data provided)
  - `Uncertainity Index`. (Source: Federal Reserve Economic Data)
  - `Gold` (Source: Macrotrends)

  


Did not use the provided JODI Demand and Supply Data, as I felt like I needed more time to understand the data better. The frequency of the data provided for some variables was not at Daily levels, hence, these datasets were merged on the level they were provided. For example, the EIA datasets were provided on a monthly level. 

### Importing the required packages

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Loading main datasets and preprocessing them. This includes ensuring Correct Column names and preparing them for merge accordingly. 


In [5]:
################# Importing the main data ######################
main_data = pd.read_excel(
    "/Users/mohammadanas/Desktop/CCI Case study/Data Science Case Study.xlsx"
)
main_data["DATE"] = pd.to_datetime(main_data["DATE"])
main_data = main_data.sort_values(by="DATE")


In [6]:
################ Importing Inflation Data #########################
inflation_data = pd.read_excel("/Users/mohammadanas/Desktop/CCI Case study/US Inflation Information.xlsx")
inflation_data['DATE'] = inflation_data['DATE'].astype('str')
# Convert the date variable to a Pandas datetime object
inflation_data['DATE'] = pd.to_datetime(inflation_data['DATE'], format='%Y%m')
inflation_data['EIA_Formatted_Date'] = inflation_data["DATE"].dt.strftime("%b-%y")
inflation_data = inflation_data.drop(columns='DATE')

In [8]:
################# Importing Uncertainity Index World #############
uncertainity_index = pd.read_csv(
    "/Users/mohammadanas/Desktop/CCI Case study/Uncertainity Index.csv"
)
uncertainity_index["DATE"] = pd.to_datetime(uncertainity_index["DATE"])
uncertainity_index["EIA_Formatted_Date"] = uncertainity_index["DATE"].dt.strftime("%b-%y")

uncertainity_index = uncertainity_index.drop(columns='DATE')

In [9]:
################# US Gasoline Prices ########################
US_gas_price = pd.read_csv(
    "/Users/mohammadanas/Desktop/CCI Case study/US Gasoline Prices.csv"
)
US_gas_price["DATE"] = pd.to_datetime(US_gas_price["DATE"])

In [10]:
################# Brent Future Prices ######################
Brent_futures = pd.read_csv(
    "/Users/mohammadanas/Desktop/CCI Case study/Brent Oil Futures Historical Data.csv"
)


Brent_futures = Brent_futures.rename(columns={"Date": "DATE", 'Price':'Brent Future', 'Vol.': 'Trade Volumes'})
Brent_futures["DATE"] = pd.to_datetime(Brent_futures["DATE"])
Brent_futures = Brent_futures.sort_values(by='DATE')

### compute average for the next ten days future prices ######
Brent_futures['Next Futures Avg'] = Brent_futures['Brent Future'].rolling(10).mean()
Brent_futures['Next Futures Avg'] = Brent_futures['Next Futures Avg'].shift(-10)


Brent_futures = Brent_futures[["DATE", "Brent Future", 'Next Futures Avg', 'Trade Volumes']]
Brent_futures = Brent_futures.dropna()

#### Preprocess trade volumes ####
Brent_futures['Trade Volumes'] = Brent_futures['Trade Volumes'].apply(lambda x: x[:-1])
Brent_futures['Trade Volumes'] = Brent_futures['Trade Volumes'].astype('float')

In [11]:
##################### Gold Prices Data #####################

gold_data = pd.read_csv(
    "/Users/mohammadanas/Desktop/CCI Case study/gold price historic data.csv"
)
gold_data = gold_data.rename(columns={"date": "DATE", " value": "Gold"})
gold_data['DATE'] = pd.to_datetime(gold_data['DATE'])
gold_data = gold_data.sort_values(by="DATE")


In [12]:
##################### Federal Funds Effective Rate #####################

FFER = pd.read_excel("/Users/mohammadanas/Desktop/CCI Case study/DFF.xlsx")

FFER_Data = FFER.rename(
    columns={
        "DFF": "DATE",
        "Federal Funds Effective Rate, Percent, Daily, Not Seasonally Adjusted": "FFER",
    }
)
FFER_Data['DATE'] = pd.to_datetime(FFER_Data['DATE']) 

In [ ]:
################ Importing Exchange Rate #########################
exchange_rate = pd.read_csv(
    "/Users/mohammadanas/Desktop/CCI Case study/exchange rate.csv", skiprows=7
)
exchange_rate.columns = exchange_rate.iloc[0]
exchange_rate = exchange_rate.iloc[1:, 0:2]
exchange_rate["Time Period"] = pd.to_datetime(exchange_rate["Time Period"])
exchange_rate = exchange_rate.loc[
    (exchange_rate["Time Period"] >= main_data["DATE"].min())
    & (exchange_rate["Time Period"] <= main_data["DATE"].max())
]
exchange_rate = exchange_rate.rename(columns={"Time Period": "DATE"})


### Loading and Preprocessing steps For Demand and Supply datasets. 

In [13]:
############# Loading OPEC and non OPEC DataSets. Contain Information on Demand and Supply ###########

OPEC_production = pd.read_csv(
    "/Users/mohammadanas/Desktop/CCI Case study/OPEC Crude Oil Production.csv",
)

NON_OPEC_production = pd.read_csv(
    "/Users/mohammadanas/Desktop/CCI Case study/Non OPEC Production.csv"
)


consumption_data = pd.read_csv(
    "/Users/mohammadanas/Desktop/CCI Case study/Consumption.csv"
)


#### Extracting the Needed Columns ####
OPEC_columns_needed = [
    "Saudi Arabia Crude Oil Production",
    "OPEC Total Crude Oil Production",
    "Total OPEC Production",
    "OPEC Total Crude Oil Production Capacity",
    "OPEC Total Surplus Crude Oil Production Capacity",
]


non_opec_production_columns = ["Total non-OPEC Liquids Petroleum Production"]


consumption_columns = [
    "Total OECD Liquid Fuels Consumption",
    "Total non-OECD Liquid Fuels Consumption",
    "Total World Liquid Fuels Consumption",
]


### EIA Datasets needed additional preprocessing. The below function acheives that. 

In [14]:
def process_EIA_data(df, column_names):
    """The function preprocesses EAI datasets as they need
    additional steps. This takes in input parameters:

    df: the dataframe to be modified
    column_names: columns to be extracted

    Outputs:
    modified dataframe"""
    df_returned = df.loc[df["Unnamed: 1"].isin(column_names)]
    df_returned.index = df_returned["Unnamed: 1"]
    df_returned = df_returned.transpose()
    df_returned = df_returned.iloc[6:]
    df_returned = df_returned.reset_index()
    df_returned.rename(columns={"index": "EIA_Formatted_Date"}, inplace=True)
    return df_returned

### Applying the function to each of the datasets. 
OPEC_production_processed = process_EIA_data(OPEC_production, OPEC_columns_needed)
NON_OPEC_production_processed = process_EIA_data(
    NON_OPEC_production, non_opec_production_columns
)
consumption_data_processed = process_EIA_data(consumption_data, consumption_columns)


### Merge Datasets into one main data. 

In [15]:
#### Creating date on Monthly level
main_data["EIA_Formatted_Date"] = main_data["DATE"].dt.strftime("%b-%y")


### Merging the data sets
final_data = main_data
all_data_sets = [Brent_futures, exchange_rate, US_gas_price, gold_data, FFER_Data]
EIA_data = [
    OPEC_production_processed,
    NON_OPEC_production_processed,
    consumption_data_processed,
    uncertainity_index,
    inflation_data
]



for i in all_data_sets:
    final_data = pd.merge(final_data, i, on="DATE", how="left")

for i in EIA_data:
    final_data = pd.merge(final_data, i, on="EIA_Formatted_Date", how="left")


In [16]:
#### Filling out missing values for some variables with previously available values. #####

################### Gasoline ######################
final_data["GASREGCOVW"] = final_data["GASREGCOVW"].ffill()
final_data["GASREGCOVW"] = final_data["GASREGCOVW"].fillna(
    US_gas_price.loc[
        US_gas_price["DATE"] <= final_data["DATE"].min(), "GASREGCOVW"
    ].iloc[-1]
)

################### Uncertainity Index ######################
final_data["WUIGLOBALWEIGHTAVG"] = final_data["WUIGLOBALWEIGHTAVG"].ffill()

################## Gold ###################
final_data['Gold'] = final_data['Gold'].ffill()

### Create Price Difference data from Brent Futures
final_data['Future_Diff'] = final_data['BRENT'] - final_data['Brent Future']


### Drop columns ###
final_data = final_data.drop(columns=["EIA_Formatted_Date", "Brent Future"])



### Export the dataframe for additional use.

In [17]:
final_data.to_csv('/Users/mohammadanas/Desktop/CCI Case study/Data for Modelling.csv')